In [2]:
# Simple S-expression lexical analysis
def slex(s):
    for p in "()": s = s.replace(p, " " + p + " ")
    return s.split()

# Simple S-expreesion syntax analysis
tnum = 0
def ssyn(s):
    global tnum
    def slist():
        global tnum
        t = s[tnum]; tnum += 1
        if   t == ')': return []
        elif t == '(': h = slist(); return [h] + slist()
        else: return [t] + slist();
    tnum = 0
    t = s[tnum]; tnum += 1
    return slist() if t == '(' else t

# Simple S-expression output function
def sdisp(x): print(x, end='', flush=True)
def swrite(s):
    def ls(x):
        swrite(x[0])
        if len(x) > 1: sdisp(' '); ls(x[1:])
    if isinstance(s, str): sdisp(s)
    elif not s: sdisp('()')
    else: sdisp('('); ls(s); sdisp(')')

# "ev" to avoid same name of Python eval
def ev(s, e):
    if   isinstance(s, str): return e[s]
    elif isinstance(s[0], str):
        if   s[0] == 'quote': return s[1]
        elif s[0] == 'atom':  return isinstance(ev(s[1], e), str)
        elif s[0] == 'car':   return ev(s[1], e)[0]
        elif s[0] == 'cdr':   return ev(s[1], e)[1:]
        elif s[0] == 'eq':    return ev(s[1], e) == ev(s[2], e)
        elif s[0] == 'cons':  return [ev(s[1], e)] + ev(s[2], e)
        elif s[0] == 'if':    return ev(s[2], e) if ev(s[1], e) else ev(s[3], e)
        else: return ev([e[s[0]]] + s[1:], e)
    else: return ev(s[0][2],
                    {**e, **dict(zip(s[0][1], [ev(x, e) for x in s[1:]]))})

# REPL without global environment
def srep(s): swrite(ev(ssyn(slex(s)), {}))

#srep("(car (cdr (quote (10 20 30)))))")

#srep(input()) # ((lambda (x) (car (cdr x))) (quote (abc def ghi)))

srep("((lambda (assoc k v) (assoc k v))                \
        (quote (lambda (k v)                             \
                (if (eq v (quote ())) (quote ())        \
                (if (eq (car (car v)) k) (car v)        \
                    (assoc k (cdr v))))))               \
        (quote Orange)                                   \
        (quote ((Apple 120) (Orange 210) (Lemon 180))))")

(Orange 210)

In [3]:
srep("(car (cdr (quote (10 20 30))))")

20

In [4]:
slex("(car (cdr (quote (10 20 30)))")

['(', 'car', '(', 'cdr', '(', 'quote', '(', '10', '20', '30', ')', ')', ')']

In [6]:
print(ssyn(slex("(car (cdr (quote (10 20 30)))))")))

['car', ['cdr', ['quote', ['10', '20', '30']]]]
